In [2]:
import tensorflow as tf 


/home/it/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/it/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/it/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/it/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: P

In [4]:
from tensorflow.examples.tutorials.mnist import input_data


In [5]:
minst = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
# helper functions 
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [7]:
# create a 2d convolution
def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [8]:
def max_pooling(x):
    # x --> [batch, H, W, Channels]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [10]:
# convolutional layer 
def conv_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [11]:
# normal layer
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])


In [13]:
x_image = tf.reshape(x, [-1, 28,28,1])

In [14]:
conv_1 = conv_layer(x_image, shape=[5,5,1,32])

In [15]:
conv_1_pooling = max_pooling(conv_1)

In [16]:
conv_2 = conv_layer(conv_1_pooling, shape=[5,5,32,64])
conv_2_pooling = max_pooling(conv_2)

In [17]:
# flatten out the result later 
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(conv_2_flat, 1024))

In [18]:
full_layer_one

<tf.Tensor 'Relu_2:0' shape=(?, 1024) dtype=float32>

In [19]:
# add a drop out 
hold_prop = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prop)

In [20]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [21]:
# loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
# optimizer 
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [23]:
train = optimizer.minimize(cross_entropy)


In [24]:
init = tf.global_variables_initializer()

In [25]:
steps = 5000

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = minst.train.next_batch(50)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prop:0.5})
        
        if i%100 == 0:
            print("On step {}".format(i))
            print("Accuracy: ")
            
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:minst.test.images, y_true:minst.test.labels, hold_prop:1.0}))
            print('\n')

On step 0
Accuracy: 


: 